In [8]:
import pandas as pd
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence

train=pd.read_csv('train.csv')

train=train.sample(frac=1)  #打乱顺序

sequence_train=train['comment_text'].values  #提取所有comments  (159571,)
test = pd.read_csv("test.csv")  
# test=test.sample(frac=1)

seqence_test = test["comment_text"].fillna('asdf').values   #????
list_classes=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y=train[list_classes].values   #(159571, 6)
print(y.shape)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

top_word=50000
max_review_len=256
word_vec_len=128


# 1. token and 词序号

tokenizer=Tokenizer(top_word)   #Tokenizer是一个用于向量化文本，或将文本转换为序列
#None或整数，处理的最大单词数量。若被设置为整数，则分词器将被限制为待处理数据集中最常见的num_words个单词
tokenizer.fit_on_texts(list(sequence_train))

sequence_train2=tokenizer.texts_to_sequences(sequence_train)
# tokenizer.fit_on_texts(list(seqence_test))
list_tokenized_test = tokenizer.texts_to_sequences(seqence_test)


'''

fit_on_texts(texts)

texts：要用以训练的文本列表

texts_to_sequences(texts)

texts：待转为序列的文本列表

返回值：序列的列表，列表中每个序列对应于一段输入文本


'''


# 2. pad_sequence
paded_sequence_train=pad_sequences(sequence_train2,max_review_len,padding='post')  #(159571, 256)
#pandding 0 的作用

X_te = pad_sequences(list_tokenized_test, max_review_len,padding='post')   #(153164, 256)

# 3. embedding
# model=Sequential()
# model.add(Embedding(mask_zero=True,input_dim=top_word,input_length=max_review_len,output_dim=word_vec_len))
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
# word2vec_train=model.predict(paded_sequence_train)
# embedding_layer=Embedding(input_dim=top_word,input_length=max_review_len,output_dim=word_vec_len)


(159571, 6)


In [9]:
X_te.shape

(153164, 256)

In [10]:
X_te.shape

(153164, 256)

In [11]:
type(sequence_train)

numpy.ndarray

In [12]:
#export CUDA_VISIBLE_DEVICES=0 
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.66
session = tf.Session(config=config)
KTF.set_session(session)

In [13]:
from keras.layers import Activation,BatchNormalization
from keras.optimizers import Adam
embedding_layer=Embedding(input_dim=top_word, input_length=max_review_len,output_dim=word_vec_len)
#嵌入层将正整数（下标）转换为具有固定大小的向量 top_word=50000 max_review_len=256 word_vec_len=128

bi_rnn=Sequential()
bi_rnn.add(embedding_layer)
bi_rnn.add(Dropout(0.2))

bi_rnn.add(BatchNormalization())
bi_rnn.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),input_shape=(max_review_len,word_vec_len)))
bi_rnn.add(GlobalMaxPool1D())
bi_rnn.add(Dropout(0.2))


bi_rnn.add(BatchNormalization())
bi_rnn.add(Dense(128))
bi_rnn.add(Dropout(0.2))

bi_rnn.add(BatchNormalization())
bi_rnn.add(Activation('relu'))
bi_rnn.add(Dense(6, activation='sigmoid'))
#bi_rnn.add(BatchNormalization())
#bi_rnn.add(Activation('si'))
bi_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
bi_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 256, 128)          6400000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256, 128)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 128)          512       
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256, 256)          263168    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
__________

In [14]:
#callback
from keras.callbacks import ModelCheckpoint
filepath="weights.best_50000.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]

bi_rnn.fit(epochs=5,batch_size=128,x=paded_sequence_train,y=y,validation_split=0.2,shuffle=True,callbacks=callbacks)



Train on 127656 samples, validate on 31915 samples
Epoch 1/5
 37248/127656 [=======>......................] - ETA: 33:53 - loss: 0.1673 - acc: 0.9421

KeyboardInterrupt: 

In [15]:
import keras
model_load=keras.models.load_model('weights.best_50000.hdf5')
y_test = model_load.predict(X_te)
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv("mul_rnn_test_50000.csv", index=False)

OSError: Unable to open file (Unable to open file: name = 'weights.best_50000.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence

train=pd.read_csv('train.csv')

train=train.sample(frac=1)

sequence_train=train['comment_text'].values
test = pd.read_csv("test.csv")
# test=test.sample(frac=1)

seqence_test = test["comment_text"].fillna('asdf').values
list_classes=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y=train[list_classes].values


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

top_word=25000
max_review_len=100
word_vec_len=100


# 1. token and 词序号

tokenizer=Tokenizer(top_word)
tokenizer.fit_on_texts(list(sequence_train))
sequence_train2=tokenizer.texts_to_sequences(sequence_train)
# tokenizer.fit_on_texts(list(seqence_test))
list_tokenized_test = tokenizer.texts_to_sequences(seqence_test)



# 2. pad_sequence
paded_sequence_train=pad_sequences(sequence_train2,max_review_len)
X_te = pad_sequences(list_tokenized_test, 100)

# 3. embedding
model=Sequential()
model.add(Embedding(mask_zero=True,input_dim=top_word,input_length=max_review_len,output_dim=word_vec_len))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
word2vec_train=model.predict(paded_sequence_train)
# word2vec_test=model.predict(X_te)

Using TensorFlow backend.


In [3]:

from keras.layers import TimeDistributed

bi_rnn=Sequential()
bi_rnn.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1),input_shape=(max_review_len,word_vec_len)))
bi_rnn.add(GlobalMaxPool1D())
bi_rnn.add( Dense(50, activation="relu"))
bi_rnn.add(Dropout(0.1))
bi_rnn.add( Dense(6, activation="sigmoid"))
bi_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
bi_rnn.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 100, 100)          60400     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
Total params: 65,756
Trainable params: 65,756
Non-trainable params: 0
_________________________________________________________________


In [2]:
from keras.layers import Activation,BatchNormalization
from keras.optimizers import Adam

bi_rnn=Sequential()
bi_rnn.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1),input_shape=(max_review_len,word_vec_len)))
bi_rnn.add(GlobalMaxPool1D())
bi_rnn.add( Dense(50))
bi_rnn.add(BatchNormalization())
bi_rnn.add(Activation('relu'))
# bi_rnn.add(Dropout(0.1))
bi_rnn.add( Dense(6))
bi_rnn.add(BatchNormalization())
bi_rnn.add(Activation('softmax'))
bi_rnn.compile(loss='binary_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
bi_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 100, 100)          60400     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
batch_normalization_1 (Batch (None, 50)                200       
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
_________________________________________________________________
batch_normalization_2 (Batch (None, 6)                 24        
__________

In [3]:
# callback
from keras.callbacks import ModelCheckpoint
filepath="weights_bi_direc-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]



bi_rnn.fit(callbacks=callbacks,epochs=1,batch_size=32,x=word2vec_train,y=y,validation_split=0.1,)
y_test = bi_rnn.predict(X_te)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv("mul_rnn_test.csv", index=False)


Train on 86265 samples, validate on 9586 samples
Epoch 1/1
86265/86265 [==============================] - 1076s - loss: 0.2275 - acc: 0.9633 - val_loss: 0.2273 - val_acc: 0.9617


ValueError: Error when checking : expected bidirectional_1_input to have 3 dimensions, but got array with shape (226998, 100)

In [9]:
import keras

model_load=keras.models.load_model('weights_bi_direc-improvement-00-0.96.hdf5')

y_test = model_load.predict(X_te)
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission[list_classes] = y_test
sample_submission.to_csv("mul_rnn_test.csv", index=False)
